In [1]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import pandas as pd
import numpy as np
import html5lib
from selenium.webdriver.common.keys import Keys
import time
from tqdm.notebook import tqdm
from selenium.common.exceptions import NoSuchElementException

In [2]:
movie_list = pd.read_excel('moviedata2.xlsx', parse_dates=['개봉일'])

In [4]:
for j in range(950, 1501, 50):
    
    movie_list = pd.read_csv('movies/movie_%d.csv' %j)
    result = {}


    for title in tqdm(movie_list['영화명']):

        options = webdriver.ChromeOptions()
        options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 적용
        options.add_argument('disable-gpu')    # GPU 사용 안함
        options.add_argument('lang=ko_KR')    # 언어 설정, options=options
        driver = webdriver.Chrome('chromedriver', options = options)
        driver.implicitly_wait(3)
        driver.get('https://www.daum.net/')
        driver.find_element_by_id('q').send_keys(title)
        driver.find_element_by_id('q').send_keys(Keys.RETURN)

        try:
            driver.find_element_by_xpath('//*[@id="movieMoreBtn"]').click()
        
            # time.sleep(1)
            # 새로운 탭으로 초점을 전환
            driver.switch_to_window(driver.window_handles[1])
            base_url = driver.current_url
            
            review_list = []
            for i in range(0, 10):
                time.sleep(1)
                an_url = '&type=netizen&page=%d' %(i+1)
                url = base_url +an_url

                res = requests.get(url)
                soup =  BeautifulSoup(res.text, 'html.parser')
                reviews = soup.find_all('p', {'class':'desc_review'})
                # print(reviews)

                for review in reviews :
                    review_list.append(review.get_text().strip())

            result[title] = review_list
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
            driver.close()
            
        except NoSuchElementException:
            driver.close()


    from pandas import DataFrame

    df = DataFrame(result.items(), columns = ['movie_id' , 'reviews'])
    df.to_csv('reviews/result_%d.csv' %(j/50), index =False, encoding='utf-8-sig') 

NameError: name 'movie_listlist' is not defined